# Importing Libraries 

In [ ]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import mediapipe as mp
import time
from datetime import datetime
import os

# Mediapipe  

In [2]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    result = model.process(image)
    return image, result

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )

# Model 

In [5]:
model = YOLO('best.pt')

# Class names

In [6]:
classnames = ['Adho', 'Alanasana', 'Anjaneyasana', 'Ardha', 'Ashta', 'Baddha', 'Bakasana', 'Balasana', 'Bandha', 
              'Bhujangasana', 'Bitilasana', 'Camatkarasana', 'Chandrasana', 'Dhanurasana', 'Eka', 'Garudasana', 
              'Halasana', 'Hanumanasana', 'Hasta', 'Kapotasana', 'Konasana', 'Malasana', 'Marjaryasana', 'Matsyendrasana', 
              'Mayurasana', 'Mukha', 'Navasana', 'One', 'Pada', 'Padangusthasana', 'Padmasana', 'Parsva', 'Parsvakonasana', 
              'Parsvottanasana', 'Paschimottanasana', 'Phalakasana', 'Pincha', 'Rajakapotasana', 'Salamba', 'Sarvangasana',
              'Setu', 'Sivasana', 'Supta', 'Svanasana', 'Svsnssana', 'Three', 'Trikonasana', 'Two', 'Upavistha', 'Urdhva', 
              'Ustrasana', 'Utkatasana', 'Uttanasana', 'Utthita', 'Vasisthasana', 'Virabhadrasana', 'Vrksasana']

# Testing rough 

In [7]:
image = cv2.imread('Padamasama.jpg')
output = model(image, show=True)
cv2.waitKey(0)

# Integrate with mediapipe 

In [9]:
cv2.rectangle??

In [ ]:
image = cv2.imread('Padamasama.jpg')
results = model(image, show=False)[0]
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    for result in results.boxes.data.tolist():
        x1,y1,x2,y2,score,class_id = result
        cv2.rectangle(image, (int(x1),int(y1)), (int(x2),int(y2)), (255,0,0), 2)
        
        image, out = mediapipe_detection(image,pose)
        
        draw_landmarks(image, out)
        plt.imshow(image[:,:,::-1])
        plt.title(classnames[int(class_id)])

In [9]:
results.names.values()

In [10]:
image = cv2.imread('Anjaneyasana.jpg')
results = model(image, show=False)[0]
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    for result in results.boxes.data.tolist():
        x1,y1,x2,y2,score,class_id = result
        cv2.rectangle(image, (int(x1),int(y1)), (int(x2),int(y2)), (255,0,0), 2)
        
        image, out = mediapipe_detection(image,pose)
        
        draw_landmarks(image, out)
        
        plt.imshow(image[:,:,::-1])
        plt.title(classnames[int(class_id)])

# Tesing with video 

In [11]:
cap = cv2.VideoCapture(0)

while True:
    
    ret, frame = cap.read()
    
    cv2.imshow('Sample', frame)
    if cv2.waitKey(1) == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [15]:
cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1800)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1200)

while True:
    
    ret, frame = cap.read()
    
    cv2.imshow('video', frame)
    
    if cv2.waitKey(1) == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [7]:
time_stamp = []

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1800)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1200)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    while True:
        
        ret, frame = cap.read()
        
        results = model(frame, show=False)[0]
        
        for result in results.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = result
            
            cv2.rectangle(frame, (int(x1),int(y1)), (int(x2),int(y2)), (0,0,255), 4)
            
            cv2.rectangle(frame, (10,10), (300,80), (255,0,0), -1)
            cv2.putText(frame, classnames[int(class_id)], (40,50), cv2.FONT_HERSHEY_DUPLEX, 1, (0,255,0), 1)
            
            try:
                if class_id:
                    now = datetime.now()
                    start_time = now.strftime("%H:%M:%S")
                    time_stamp.append(start_time)

                    t1 = datetime.strptime(time_stamp[0], "%H:%M:%S")

                    t2 = datetime.strptime(time_stamp[-1], "%H:%M:%S")

                    delta = (t2 - t1)
                    df = "Hold this Exercise position for "+str(8-int(delta.total_seconds()))+" sec."
                    if 8-int(delta.total_seconds())<0:
                        df = "YOU COMPLETED THIS SET"
                    
                    cv2.rectangle(frame, (1500,12),(900,73),(23,210,28),-1)
                    cv2.putText(frame,str(df) ,(920,40),cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                
                else:
                    time_stamp = []
            
            except:
                pass
                
            image, out = mediapipe_detection(frame,pose)
        
            draw_landmarks(image, out)
            
            cv2.imshow('Output', frame)
            
        if cv2.waitKey(1) == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

In [10]:
cap.release()
cv2.destroyAllWindows()

In [7]:
# time_stamp = []

# count = 2

# yoga = 'Adho'

# newName = ['Adho', 'Alanasana', 'Anjaneyasana', 'Ardha', 'Ashta', 'Baddha', 'Bakasana', 'Balasana', 'Bandha', 
#               'Bhujangasana', 'Bitilasana', 'Camatkarasana', 'Chandrasana', 'Dhanurasana', 'Eka', 'Garudasana', 
#               'Halasana', 'Hanumanasana', 'Hasta', 'Kapotasana', 'Konasana', 'Malasana', 'Marjaryasana', 'Matsyendrasana', 
#               'Mayurasana', 'Mukha', 'Navasana', 'One', 'Pada', 'Padangusthasana', 'Parsva', 'Parsvakonasana', 
#               'Parsvottanasana', 'Paschimottanasana', 'Phalakasana', 'Pincha', 'Rajakapotasana', 'Salamba', 'Sarvangasana',
#               'Setu', 'Sivasana', 'Supta', 'Svanasana', 'Svsnssana', 'Three', 'Trikonasana', 'Two', 'Upavistha', 'Urdhva', 
#               'Ustrasana', 'Utkatasana', 'Uttanasana', 'Utthita', 'Vasisthasana', 'Virabhadrasana', 'Vrksasana']

# cap = cv2.VideoCapture(0)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1800)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1200)

# with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
#     while True:
        
#         ret, frame = cap.read()
        
#         results = model(frame, show=False)[0]
        
#         for result in results.boxes.data.tolist():
#             x1, y1, x2, y2, score, class_id = result
            
#             cv2.rectangle(frame, (int(x1),int(y1)), (int(x2),int(y2)), (0,0,255), 4)
            
#             cv2.rectangle(frame, (10,10), (300,80), (255,0,0), -1)
#             #cv2.putText(frame, classnames[int(class_id)], (40,50), cv2.FONT_HERSHEY_DUPLEX, 1, (0,255,0), 1)
#             cv2.putText(frame, yoga, (40,50), cv2.FONT_HERSHEY_DUPLEX, 1, (0,255,0), 1)
            
#             try:
#                 if yoga in newName:
#                     now = datetime.now()
#                     start_time = now.strftime("%H:%M:%S")
#                     time_stamp.append(start_time)
#                     t1 = datetime.strptime(time_stamp[0], "%H:%M:%S")

#                     t2 = datetime.strptime(time_stamp[-1], "%H:%M:%S")

#                     delta = (t2 - t1)
#                     print(delta)
#                     for i in range(count):
#                         cv2.rectangle(frame, (1500, 25), (1000, 140), (0, 0, 255), -1)
#                         cv2.rectangle(frame, (1500, 12), (900, 73), (23, 210, 28), -1)
#                         for sec in range(8, 0, -1):
#                             df = "Hold this Exercise position for " + str(sec) + " sec."
#                             cv2.putText(frame, str(df), (920, 40), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
#                             dfs = f"YOU COMPLETED {i} SET"
#                             cv2.putText(frame, str(dfs), (1050, 110), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
#                             print(i)
            
                
#                 else:
#                     time_stamp = []
            
#             except:
#                 pass
                
#             image, out = mediapipe_detection(frame,pose)
        
#             draw_landmarks(image, out)
            
#             cv2.imshow('Output', frame)
            
#         if cv2.waitKey(1) == ord('q'):
#                 break

# cap.release()
# cv2.destroyAllWindows()

In [22]:
time_stamp = []

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1800)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1200)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    while True:
        
        ret, frame = cap.read()
        
        results = model(frame, show=False)[0]
        
        for result in results.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = result
            
            cv2.rectangle(frame, (int(x1),int(y1)), (int(x2),int(y2)), (0,0,255), 4)
            
            cv2.rectangle(frame, (10,10), (300,80), (255,0,0), -1)
            cv2.putText(frame, classnames[int(class_id)], (40,50), cv2.FONT_HERSHEY_DUPLEX, 1, (0,255,0), 1)
            
                
            image, out = mediapipe_detection(frame,pose)
        
            draw_landmarks(image, out)
            
            cv2.imshow('Output', frame)
            
        if cv2.waitKey(1) == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()